# Alpha Decay - Quand faut-il arrêter de trader une stratégie rentable ?

**Alpha :**  
$$\large \alpha = \mathbb{E}[r_s] - \mathbb{E}[r_m]$$

où l'espérance des rendements est déterminée par :
  
$$\large \mathbb{E}[r_s] = p \cdot G - (1 - p) \cdot L$$

avec :
- $p$ : probabilité de gain  
- $G$ : gain moyen  
- $L$ : perte moyenne
- $r_m$ : benchmark



Modélisation stochastique de l'Alpha Decay

**Alpha à l'instant $t$ :**

$$\alpha(t) = p_t \cdot G_t - (1 - p_t) \cdot L_t - \mathbb{E}[r_m]$$

avec :  
- $p_t = \frac{1}{1 + e^{-x_t}}$ avec $dx_t = \theta (\mu - x_t) \, dt + \sigma \, dW_t$ 
- $G_t = L_t$

$$\large \mathbb{E}[r_s] > 0$$
$$\large p \cdot G - (1 - p) \cdot L > 0$$
$$\large G \cdot (2p - 1) > 0$$
$$\large 2p - 1 > 0$$
$$\large p > \frac{1}{2}$$

In [1]:
import numpy as np
import plotly.graph_objects as go

T = 500
dt = 1
theta = 0.01
mu = 0.05
sigma = 0.01

x = np.zeros(T)
p = np.zeros(T)
x[0] = 0.7
p[0] = 1 / (1 + np.exp(-x[0]))

for t in range(1, T):
    dx = theta * (mu - x[t - 1]) * dt + sigma * np.sqrt(dt) * np.random.normal()
    x[t] = x[t - 1] + dx
    p[t] = 1 / (1 + np.exp(-x[t]))


In [2]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=p, name="p(t)"))
fig.add_trace(go.Scatter(y=1 - p, name="1 - p(t)"))

fig.update_layout(
    title="Évolution stochastique de la probabilité de gain et de perte",
    xaxis_title="Temps",
    yaxis_title="Probabilité",
    template="plotly_dark",
    #yaxis=dict(range=[0, 1]),
    width=800
)

fig.show()
print(f"Moyenne de probabilité de gain : {p.mean():.2f}")


Moyenne de probabilité de gain : 0.54


In [3]:
G = 10
L = 10
T0 = 1000
n_sim = 50

final_equities = np.zeros(n_sim)

fig_mc = go.Figure()

for sim in range(n_sim):
    equity = np.zeros(T)
    returns = np.zeros(T)
    equity[0] = T0

    for t in range(1, T):
        returns[t] = G if np.random.rand() < p[t] else -L
        equity[t] = equity[t - 1] + returns[t]

    final_equities[sim] = equity[-1]
    fig_mc.add_trace(
        go.Scatter(
            y=equity, mode='lines', line=dict(width=1), showlegend=False
        )
    )


In [4]:
fig_mc.update_layout(
    title="Monte Carlo des Equity Curves (avec probas dynamiques)",
    xaxis_title="Temps",
    yaxis_title="Capital",
    template="plotly_dark",
    width=800,
    height=500
)

fig_mc.show()
mean_final = final_equities.mean()
print(f"Moyenne du capital final : {mean_final:.2f}")

Moyenne du capital final : 1403.20


In [5]:
rm = 0.001

equity = np.zeros(T)
equity[0] = T0

for t in range(1, T):
    r = G if np.random.rand() < p[t] else -L
    equity[t] = equity[t - 1] + r

rm_curve = T0 * (1 + rm)**np.arange(T)


In [6]:
fig_comp = go.Figure()

fig_comp.add_trace(
    go.Scatter(y=equity, name="Equity Curve (stratégie)", line=dict(width=2))
)

fig_comp.add_trace(
    go.Scatter(
        y=rm_curve, name="Benchmark (r_m)", line=dict(width=2, dash='dash')
    )
)

fig_comp.update_layout(
    title="Stratégie vs Benchmark",
    xaxis_title="Temps",
    yaxis_title="Capital",
    template="plotly_dark",
    yaxis_type="log",
    width=800,
    height=500
)

fig_comp.show()


In [7]:
alpha = equity - rm_curve


In [8]:
fig_alpha = go.Figure()

fig_alpha.add_trace(
    go.Scatter(
        y=alpha, name="Alpha cumulé (Equity - Benchmark)", line=dict(width=2)
    )
)

fig_alpha.update_layout(
    title="Alpha cumulé",
    xaxis_title="Temps",
    yaxis_title="Spread de capital",
    template="plotly_dark",
    width=800,
    height=500
)

fig_alpha.show()
